## $\S$1. Introduction
#### What is Apache Flink?
* Flink是工作在无界(Unbounded streams)和有界数据流(Bounded streams)上的计算引擎  
* 所有种类的数据, 都能被看做由事件流(stream of events)产生的: 像信用卡交易,机器日志,用户在手机和网页上的交互动作等
* Unbounded streams: 
    * 无界流数据要被迅速处理, 不等企图等待所有数据到达以后再处理, 因为无界数据流不会停止, 有开始, 无结束
    * 处理无界数据需要事件(event)以某种顺序抵达
* Bounded streams:
    * 指通常的批处理, 有开始和结束
    * 有界数据流不需要事件按顺序到达, 因为有界数据可能已经被排序
* Flink精通于同时处理有界和无界数据. Flink对时间和状态的精确控制, 使得其可在无界数据流上运行各种应用;  
  同时专门为有界数据流设计了数据结构和算法来加速处理
  
#### Run Applications at any Scale
* Flink的异步性, 和持续增加的chaeck point机制, 会使得处理延迟带来的影响到最小, 保证处理数据是exactly-once一致性

#### Building Blocks for Streaming Applications
流处理框架主要看如何处理streams, state, time; 下面依次介绍
* Streams
    * Bounded and unbounded streams: Flink可以处理有界和无界数据
    * Real-time and recorded streams: Flink可以对数据进行实时处理和先记录再处理
* State
* Time
    * 重要的是, 应用如何处理event-time和processing-time
    * Event-time Mode:
    * Watermark Support:
    * Late Data Handling:
    * Processing-time Mode:
    
#### DeployMode
* **Start a Local Flink Cluster**  

```bash 
$ ./bin/start-cluster.sh  # Start Flink
$ ./bin/stop-cluster.sh   # Stop Flink
```

* **Create YARN Session**  
 YARN session是在Hadoop YARN环境下启动一个Flink cluster集群，里面的资源是可以共享给其他的Flink作业。 
```bash  
$ ./bin/yarn-session.sh -n 4 -tm 8192 -s 8
 # 上面命令启动了4个TaskManager，每个TaskManager内存为8G且占用了8个核
```

```bash
$ ./bin/flink run ./examples/batch/WordCount.jar
```

* **Run a single Flink job on YARN**   
 我们还可以在YARN上启动一个Flink作业。这里我们还是使用./bin/flink，但是不需要事先启动YARN session：
```shell
$ ./bin/flink run -m yarn-cluster -yn 2 ./examples/batch/WordCount.jar 
```

* **Scala REPL**
```shell
bin/start-scala-shell.sh local
bin/start-scala-shell.sh yarn -n 2 # Yarn Scala Shell cluster
bin/start-scala-shell.sh yarn      # Yarn session
```

##  $\S$2.应用场景

[https://flink.apache.org/zh/usecases.html](https://flink.apache.org/zh/usecases.html)
* 事件驱动型应用
* 数据分析应用
* 数据管道应用


##  $\S$3. Concepts
#### Dataflow Programming Model
* **Programs and Dataflows**
    * Flink程序都会被映射成streaming dataflows,由streams和operators组成 (a stream is a (potentially never-ending) flow of data records);   
    输入称为connector,输出称为sink
    * Parallel Dataflows
        * Flink的operator分为两类(对比spark的款依赖, 窄依赖)
            * One-to-one streams
            * Redistributing streams 
    * [parallel execution](https://ci.apache.org/projects/flink/flink-docs-release-1.8/dev/parallel.html)  
* **Windows(窗口)**    
    * 流处理的Aggregating操作与批处理不同, 因为流处理没有bound, 所以要指定流处理的window, 作为聚合操作的作用范围
    * window可以分为两种: 面向时间(time)的, 也可以是面向数据(data)的.  
      例如: “count over the last 5 minutes”, or “sum of the last 100 elements”
    * 从另一个角度讲, window还可以被划分为3种: 
        * 滚动窗口: tumbling windows (窗口之间无交叉)
        * 滑动窗口: sliding windows (窗口之间有交叉)
        * session窗口: 被不活动的时间间隔切割成窗口
    * ref
        * [blog](https://flink.apache.org/news/2015/12/04/Introducing-windows.html)
        * [docs](https://ci.apache.org/projects/flink/flink-docs-release-1.8/dev/stream/operators/windows.html)
* **Time** *
    * 当谈及流式处理中time(时间)时, 通常在讨论三种时间
        * Event time: event被创建的时间,通常是一个时间戳, 由生产者产生
        * Ingestion time: event进入flink dataflow的时间
        * Processing time: 每个flink节点处理数据的本地时间
    * [doc](https://ci.apache.org/projects/flink/flink-docs-release-1.8/dev/event_time.html)
* **Stateful Operations** 
    * Flink中的许多操作都是针对单个事件的, 比如event parser. 但有些操作需要跨多个事件, 例如window operators, 这种操作称为stateful
    * stateful operations的状态可以被认为是存在一个嵌入式的key/value存储里, 这些状态会被重新分区并分不到多台机器中.  
      使用`keyBy()`方法后, 会根据事件的key重新分布事件, 让后续的更新操作为本地操作
* **Checkpoints实现容错**
    * flink使用流回放(streaming replay)和检查点(checkpoint)实现容错
       

##  $\S$4. Distributed Runtime Environment

* **Job Managers, Task Managers, Clients**
    * Job Managers: mster的角色, 负责调度任务, 协调checkpoint等
    * Task Managers: worder角色, 负责执行任务, buffer和exchange数据流
    * Clients: client不是flink运行时的一部分,只负责准备和发送数据流到Job Manager, 此后就可以断开与flink的连接,也可以持续保持连接等待报告返回
* **Task Slots and Resources**
    * task slots: 
        * 每个worker都是一个jvm进程, 可以启用1个或多个线程执行task.
        * slot数量表示每个worker启动几个线程处理子任务, 默认每个线程处理一个子任务, 每个线程均分worker占用的内存, worker至少有一个slot
* **State Backends**
    *  key/values的存储形式取决于flink选用的state后端. 可选择的后端有内存中的hash-map结构, 和嵌入式kv数据库RocksDB
    * state后端除了负责存储key/values, 同时也负责对key/values左check-point
    
* **Savepoints**
    * Savepoints是手动触发的checkpoint, 可以让datastream下次从该位置继续执行
    * Savepoints与checkpoint的区别是: Savepoints需要手动触发, 且不会自动失效(当新的checkpoint产生后)
